# Test

In [1]:
%load_ext autoreload
%autoreload 2

Import datasets using functions from src/data/data.py. Datasets are downloaded from huggingface and stored in /data. Once downloaded, datasets are loaded locally.

Run ```pip install -e .``` if module importing isn't working.

In [5]:
from src.data.data import get_in_domain, get_out_domain
from src.data.utils import get_random_subsets

in_domain = get_in_domain()
out_domain = get_out_domain()

print(f"In domain:\n{in_domain}")
print(in_domain[0])

print(f"Out of domain:\n{out_domain}")
print(out_domain[10])

# get_random_subsets(in_domain)

In domain:
Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 261802
})
{'premise': 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him', 'hypothesis': 'You lose the things to the following level if the people recall.', 'label': 0, 'idx': 1}
Out of domain:
Dataset({
    features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
    num_rows: 10000
})
{'premise': 'The president avoided the athlete .', 'hypothesis': 'The athlete avoided the president .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NN president)) (VP (VBD avoided) (NP (DT the) (NN athlete))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NN at

{2: [Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  }),
  Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  }),
  Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  }),
  Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  }),
  Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  }),
  Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  }),
  Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  }),
  Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  }),
  Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  }),
  Dataset({
      features: ['premise', 'hypothesis', 'label', 'idx'],
      num_rows: 2
  })],
 16: [Dataset({
      features: ['premise', 'h

Import models using methods from src/models/opt.py. Models are downloaded from huggingface and stored in /models/pretrained. Once downloaded, models are loaded locally.

In [3]:
from src.model.model import get_model

tokenizer_opt125, model_opt125 = get_model('opt-125m')
tokenizer_opt350, model_opt350 = get_model('opt-350m')

Few-shot finetuning.

In [4]:
from src.finetuners.fewshot import fine_tune

train_dataset = in_domain.select(range(2))
eval_dataset = out_domain.select(range(2))

fine_tune(model=model_opt125, tokenizer=tokenizer_opt125, train_dataset=train_dataset, eval_dataset=eval_dataset)

100%|██████████| 40/40 [00:10<00:00,  3.86it/s]


{'train_runtime': 10.3669, 'train_samples_per_second': 7.717, 'train_steps_per_second': 3.858, 'train_loss': 0.061122357845306396, 'epoch': 40.0}


100%|██████████| 1/1 [00:03<00:00,  3.09s/it]


{'accuracy': 0.0,
 'total_inference_time': 3.1549,
 'average_inference_time_per_sample': 1.57745,
 'peak_memory_usage_gb': 1.5613317489624023}